In [5]:
import pandas as pd
df = pd.read_csv("sampled_data_1233455.csv")

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

features = ["0","1", "2", "3", "4", "5", "7"]
X = df[features]
y = df["8"]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

pca_df = pd.DataFrame(data = X_pca, columns = ["Feature 1", "Feature 2"])
dataset = pd.concat([pca_df, y], axis=1)
dataset=dataset.rename(columns={"8":"Class Label"})
dataset.head()

,Feature 1,Feature 2,Class Label
0,0.448372,-0.903387,0.0
1,-0.213044,0.550000,0.0
2,0.900868,-0.010139,1.0
3,0.374398,-0.850995,0.0
4,0.663910,-0.437368,0.0


In [7]:
X = dataset[['Feature 1', 'Feature 2']].values
y = dataset['Class Label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import TrainableFidelityQuantumKernel
from qiskit.utils import QuantumInstance
from qiskit import BasicAer

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=42, seed_transpiler=42)
feature_map = ZZFeatureMap(feature_dimension=2, reps=2, entanglement='linear')
quantum_kernel = TrainableFidelityQuantumKernel(feature_map=feature_map)
qsvc = QSVC(quantum_kernel=quantum_kernel)

/var/folders/j5/xjgdn5n12cv8xt7g7_l1vp280000gn/T/ipykernel_11705/1430067715.py:9: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=42, seed_transpiler=42)


In [9]:
qsvc.fit(X_train, y_train)
y_pred = qsvc.predict(X_test)

In [10]:
from sklearn.metrics import classification_report
train_predictions = qsvc.predict(X_train)
test_predictions = qsvc.predict(X_test)

report_train = classification_report(y_train, train_predictions, target_names=['Class 0', 'Class 1'])
print("Classification Report for Training Data:")
print(report_train)
report_test = classification_report(y_test, test_predictions, target_names=['Class 0', 'Class 1'])
print("Classification Report for Testing Data:")
print(report_test)


Classification Report for Training Data:
              precision    recall  f1-score   support

     Class 0       0.65      0.50      0.57       532
     Class 1       0.58      0.71      0.64       508

    accuracy                           0.61      1040
   macro avg       0.61      0.61      0.60      1040
weighted avg       0.61      0.61      0.60      1040

Classification Report for Testing Data:
              precision    recall  f1-score   support

     Class 0       0.62      0.51      0.56       130
     Class 1       0.58      0.69      0.63       130

    accuracy                           0.60       260
   macro avg       0.60      0.60      0.60       260
weighted avg       0.60      0.60      0.60       260

